In [ ]:
用語説明
遺伝的アルゴリズムで使う用語は、一般的な最適化アルゴリズムで使う用語と少し異なるので確認しておきます。機械用語が生物用語に置き替わります。

遺伝子(gene) : 一つの設計変数のこと。
個体(individual) : 設計変数の1セット。
個体集合(population) : 個体を集めたセット。現世代(population)と次世代(offspring)の2つを用意する必要があります。
世代(generation) : 現世代と次世代を包括した個体集合の表現。
適応度(fitness) : 各個体に対する目的関数の値。
選択(selection) : 現世代から次世代への淘汰のこと。適応度の高いものを優先的に選択します。
交叉(crossover) : 2個体間の遺伝子の入れ替えのこと。生物が交配によって子孫を残すことをモデル化したもの。
突然変異(mutation) : 個体の遺伝子をランダムに変化させること。
英語でカッコ書きしたのは、プログラムを書くときに変数名に使いたいからです。

アルゴリズムの流れを、ステップ別で見ていきましょう。

ステップ1(初期世代の作成)
先ずはN個の個体をランダムに生成し、最初の現世代となる個体集合を作ってやります。

その後、それぞれの個体で適応度を計算しておきます。

ステップ2 (選択)
現世代からランダムにNt個の個体を取り出し、一番適応度が高い個体を1つ選び空の次世代プールに追加します。次世代の個体数が現世代と同じN個になるまでこの操作を繰り返します。

この方法はトーナメント選択と呼ばれており、Ntをトーナメントサイズと呼びます。

ステップ3 (交叉)
ステップ2で作成した次世代プール中の個体を2つのグループに分割します。グループ分けした後の次世代プールは空にしておきます。

それぞれのグループで1個体ずつ、計2個体取り出し、確率Pcでこの2個体同士で遺伝子の入れ替えを行い、空の次世代プールに戻していきます。この確率Pcの事を交叉率と呼びます。

交叉が発生しない場合は、そのまま2個体を次世代プールに戻す形になります。

遺伝子の入れ替え方法は、個体の遺伝子列内にランダムで交叉点を二つ入れてやり、二つの交叉点に挟まれている部分を入れ換えてやります。2点交叉と呼ばれています。

ステップ4 (突然変異)
ステップ3でアップデートした次世代プールから確率Pmi (個体突然変異率)で突然変異が発生する個体を選びます。

選んだ個体の各遺伝子を確率Pmg (遺伝子突然変異率)でランダムに変更します。

その後、次世代を現世代に上書きしステップ2に戻ります。

ステップ2からステップ4を繰り返すといつの間にか最適解が求まるというわけです。

In [1]:
import random
import numpy as np
from operator import attrgetter

def main():
    n_gene   = 100   # The number of genes.
    n_ind    = 300   # The number of individuals in a population.
    CXPB     = 0.5   # The probability of crossover.
    MUTPB    = 0.2   # The probability of individdual mutation.
    MUTINDPB = 0.05  # The probability of gene mutation.
    NGEN     = 40    # The number of generation loop.

    random.seed(64)
    # --- Step1 : Create initial generation.
    pop = create_pop(n_ind, n_gene)
    set_fitness(evalOneMax, pop)
    best_ind = max(pop, key=attrgetter("fitness"))
    
    # --- Generation loop.
    print("Generation loop start.")
    print("Generation: 0. Best fitness: " + str(best_ind.fitness))
    for g in range(NGEN):
        
        # --- Step2 : Selection.
        offspring = selTournament(pop, n_ind, tournsize=3)
        
        # --- Step3 : Crossover.
        crossover = []
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                child1, child2 = cxTwoPointCopy(child1, child2)
                child1.fitness = None
                child2.fitness = None
            crossover.append(child1)
            crossover.append(child2)

        offspring = crossover[:]
        
        # --- Step4 : Mutation.
        mutant = []
        for mut in offspring:
            if random.random() < MUTPB:
                mut = mutFlipBit(mut, indpb=MUTINDPB)
                mut.fitness = None
            mutant.append(mut)

        offspring = mutant[:]
        
        # --- Update next population.
        pop = offspring[:]
        set_fitness(evalOneMax, pop)
        
        # --- Print best fitness in the population.
        best_ind = max(pop, key=attrgetter("fitness"))
        print("Generation: " + str(g+1) + ". Best fitness: " + str(best_ind.fitness))
    
    print("Generation loop ended. The best individual: ")
    print(best_ind)
        
class Individual(np.ndarray):
    """Container of a individual."""
    fitness = None
    def __new__(cls, a):
        return np.asarray(a).view(cls)

def create_ind(n_gene):
    """Create a individual."""
    return Individual([random.randint(0, 1) for i in range(n_gene)])

def create_pop(n_ind, n_gene):
    """Create a population."""
    pop = []
    for i in range(n_ind):
        ind = create_ind(n_gene)
        pop.append(ind)
    return pop

def set_fitness(eval_func, pop):
    """Set fitnesses of each individual in a population."""
    for i, fit in zip(range(len(pop)), map(eval_func, pop)):
        pop[i].fitness = fit
        
def evalOneMax(ind):
    """Objective function."""
    return sum(ind)

def selTournament(pop, n_ind, tournsize):
    """Selection function."""
    chosen = []    
    for i in range(n_ind):
        aspirants = [random.choice(pop) for j in range(tournsize)]
        chosen.append(max(aspirants, key=attrgetter("fitness")))
    return chosen

def cxTwoPointCopy(ind1, ind2):
    """Crossover function."""
    size = len(ind1)
    tmp1 = ind1.copy()
    tmp2 = ind2.copy()
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size-1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    tmp1[cxpoint1:cxpoint2], tmp2[cxpoint1:cxpoint2] = tmp2[cxpoint1:cxpoint2].copy(), tmp1[cxpoint1:cxpoint2].copy()
    return tmp1, tmp2

def mutFlipBit(ind, indpb):
    """Mutation function."""
    tmp = ind.copy()
    for i in range(len(ind)):
        if random.random() < indpb:
            tmp[i] = type(ind[i])(not ind[i])
    return tmp

if __name__ == "__main__":
    main()


Generation loop start.
Generation: 0. Best fitness: 64
Generation: 1. Best fitness: 66
Generation: 2. Best fitness: 68
Generation: 3. Best fitness: 68
Generation: 4. Best fitness: 71
Generation: 5. Best fitness: 73
Generation: 6. Best fitness: 74
Generation: 7. Best fitness: 76
Generation: 8. Best fitness: 76
Generation: 9. Best fitness: 80
Generation: 10. Best fitness: 82
Generation: 11. Best fitness: 82
Generation: 12. Best fitness: 84
Generation: 13. Best fitness: 84
Generation: 14. Best fitness: 86
Generation: 15. Best fitness: 88
Generation: 16. Best fitness: 88
Generation: 17. Best fitness: 88
Generation: 18. Best fitness: 90
Generation: 19. Best fitness: 91
Generation: 20. Best fitness: 91
Generation: 21. Best fitness: 91
Generation: 22. Best fitness: 92
Generation: 23. Best fitness: 92
Generation: 24. Best fitness: 94
Generation: 25. Best fitness: 95
Generation: 26. Best fitness: 96
Generation: 27. Best fitness: 96
Generation: 28. Best fitness: 97
Generation: 29. Best fitness: 